In [1]:
import re
import requests
import unicodedata
import pprint
from bs4 import BeautifulSoup
import sys
import json
from helpers import *

In [2]:
endpoint = "https://www.sec.gov/cgi-bin/browse-edgar"

'''
Stocks tried so far
NVDA
AMD
PLUG - next to work on
'''

param_dict = {
#     'action': 'getcompany',
    'owner': 'exclude',
    'output': 'atom',
    'CIK': 'NVDA',
    'type': '10-K'
}

resp = requests.get(url = endpoint, params = param_dict)

In [3]:
soup = BeautifulSoup(resp.content, 'lxml')
master_list_xml = fetch_master_list(soup)

# Note: Only fetches reports with interactive data
annual_reports = dict(filter(lambda el: el[1]['category']['term'] == '10-K' and el[1]['file_info']['xbrl_href'] != None, master_list_xml.items()))

links_text = {}
with open('./links_text.json') as f:
    links_text = json.load(f)

In [7]:
%%time

results = []

data = annual_reports[list(annual_reports.keys())[0]]
year = int(re.search('^\d+', data['file_info']['filing_date']).group())
while year >= 2012:
    url = data['file_info']['xbrl_href']
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    script = soup.find('script', src=None)
    table_links = re.findall('/Archives/edgar/data/.+R.+\.htm', script.string)
    links = fetch_statement_links(soup, table_links, links_text)
    metrics = parse_statements(links, links_text)
        
    results.append(metrics)
    
    if len(results) > 0:
        year, data = get_next_report(results, annual_reports)

pp = pprint.PrettyPrinter()
pp.pprint(results)

[{'CONSOLIDATED BALANCE SHEETS': {'Long-term debt': ['1,991\n', '1,988\n'],
                                  'Total liabilities': ['5,111\n', '3,950\n'],
                                  "Total liabilities and shareholders' equity": ['$ '
                                                                                 '17,315\n',
                                                                                 '$ '
                                                                                 '13,292\n'],
                                  "Total shareholders' equity": ['12,204\n',
                                                                 '9,342\n'],
                                  'dates': ['Jan. 26, 2020', 'Jan. 27, 2019'],
                                  'statement_header': 'CONSOLIDATED BALANCE '
                                                      'SHEETS - USD ($) $ in '
                                                      'Millions'},
  'CONSOLIDATED STATEMENTS OF

In [ ]:
'''
TODO:
- Test out with more stocks
- Convert json file with text patterns into regex
- Parse / clean up results dict
'''